# NLP

## Design Statement
Design a conversational journaling assistant to enable young adults in a mental health self-reflection context to express and understand their emotions with personalized sentiment feedback and journaling prompts.

## Techincal Architecture
### System Diagram
A[User Journal Entry] --> B[LangChain LLMChain]     
B --> C[Sentiment Reflection Generator]     
B --> D[Follow-up Prompt Generator]     
B --> E[Memory Storage (FAISS)]     
C --> F[Streamlit UI Output]        
D --> F[Streamlit UI Output]        

### Components overview
| Component       | Tool                                         | Description                                                            |
| --------------- | -------------------------------------------- | ---------------------------------------------------------------------- |
| LLM             | Ollama                              | LLM model for sentiment + response generation                          |
| LangChain       | `LLMChain`, `PromptTemplate`, `SimpleMemory` | Core logic chaining inputs and outputs                                 |
| Streamlit       | Frontend                                     | UI with textbox, buttons, and results                                  |
|  Memory | Chroma / FAISS                               | Store past entries and provide history-aware feedback  |


# Steps taken
1. Created basic Streamlit UI and added sentiment reflection using the llama3 model
2. Created follow-up prompt generation for better self-reflection
3. Implemented history with FAISS
4. Implemented displaying of past reflections

## Streamlit App Code
the code can be found in app.py